## Mapping data from ICOADS deck 704 to the Common Data Model (CDM)

Here we extract supplemental metadata from [ICOADSv3.0](https://icoads.noaa.gov/r3.html) stored in the [IMMA version 1](https://icoads.noaa.gov/e-doc/imma/R3.0-imma1.pdf) format. 
We will then map this data (including the supplemental data) to the Common Data Model (CDM) format defined in the [CDM Documentation](https://github.com/glamod/common_data_model/blob/master/cdm_latest.pdf)..

This is done by using the `cdm_mapper` component of the `cdm_reader_mapper` package. The supplementary data are mapped to the CDM using the [tables](https://github.com/glamod/cdm_reader_mapper/tree/main/cdm_reader_mapper/cdm_mapper/tables/icoads/r300/d704) and [codes](https://github.com/glamod/cdm_reader_mapper/tree/main/cdm_reader_mapper/cdm_mapper/codes/icoads/r300/d704) specific to deck 704. The generic ICOADS [tables](https://github.com/glamod/cdm_reader_mapper/tree/main/cdm_reader_mapper/cdm_mapper/tables/icoads) are used to map the common ICOADS data components.

We are analysing deck: `704`, the [US Marine Meteorological Journals Collection](https://icoads.noaa.gov/usmmj.html)

In [1]:
from __future__ import annotations

import os
import sys

import pandas as pd

import cdm_reader_mapper.cdm_mapper as cdm
from cdm_reader_mapper import data, mdf_reader

2024-10-09 07:50:33,352 - root - INFO - init basic configure of logging success


We first read the supplemental data information from the `c99` imma format for a subset of the data (e.g. 1878/10). For this we need to use the `"icoads_r300_d704"` schema. The convention for schema names is: `"format_version_deck"`

* format/data model: "icoads"
* version/release: "r300" (release 3.0.0)
* deck: "d704"

In this notebook we load the icoads r3.0.0 deck 704 test file to use as an example.

In [2]:
schema = "icoads_r300_d704"

data_file_path = data.test_data.test_icoads_r300_d704[
    "source"
]  # Load the example file from the cdm_reader_mapper test data
data_raw = mdf_reader.read(data_file_path, imodel=schema)

2024-10-09 07:50:36,685 - root - INFO - Attempting to fetch remote file: icoads/r300/d704/input/icoads_r300_d704_1878-10-01_subset.imma.md5
2024-10-09 07:50:36,792 - root - INFO - READING DATA MODEL SCHEMA FILE...
2024-10-09 07:50:36,799 - root - INFO - EXTRACTING DATA FROM MODEL: icoads_r300_d704
2024-10-09 07:50:36,799 - root - INFO - Getting data string from source...
2024-10-09 07:50:36,962 - root - WARNING - Data numeric elements with missing upper or lower threshold: ('c1', 'BSI'),('c1', 'AQZ'),('c1', 'AQA'),('c1', 'UQZ'),('c1', 'UQA'),('c1', 'VQZ'),('c1', 'VQA'),('c1', 'PQZ'),('c1', 'PQA'),('c1', 'DQZ'),('c1', 'DQA'),('c5', 'OS'),('c5', 'OP'),('c5', 'FM'),('c5', 'IMMV'),('c5', 'IX'),('c5', 'W2'),('c5', 'WMI'),('c5', 'SD2'),('c5', 'SP2'),('c5', 'IS'),('c5', 'RS'),('c5', 'IC1'),('c5', 'IC2'),('c5', 'IC3'),('c5', 'IC4'),('c5', 'IC5'),('c5', 'IR'),('c5', 'RRR'),('c5', 'TR'),('c5', 'NU'),('c5', 'QCI'),('c5', 'QI1'),('c5', 'QI2'),('c5', 'QI3'),('c5', 'QI4'),('c5', 'QI5'),('c5', 'QI6')

The data from the c99 column for this deck is separated in the following sub sections:
- c99_sentinal
- c99_journal
- c99_voyage
- c99_daily
- c99_data4
- c99_data5

In [3]:
data_raw.data.c99_sentinal.head()

,ATTI,ATTL,BLK
0,99,0,NaN
1,99,0,NaN
2,99,0,NaN
3,99,0,NaN
4,99,0,NaN


In [4]:
pd.options.display.max_columns = None
data_raw.data.c99_journal.head()

,sentinal,reel_no,journal_no,frame_no,ship_name,journal_ed,rig,ship_material,vessel_type,vessel_length,vessel_beam,commander,country,screw_paddle,hold_depth,tonnage,baro_type,baro_height,baro_cdate,baro_loc,baro_units,baro_cor,thermo_mount,SST_I
0,1,002,0018,0003,Panay,78,01,1,1,187,37,"S.P.Bray,Jr",01,3,23,1190,2,14,NaN,Bulkhead of cabin,1,- .102,2,NaN
1,1,002,0018,0003,Panay,78,01,1,1,187,37,"S.P.Bray,Jr",01,3,23,1190,2,14,NaN,Bulkhead of cabin,1,- .102,2,NaN
2,1,002,0018,0003,Panay,78,01,1,1,187,37,"S.P.Bray,Jr",01,3,23,1190,2,14,NaN,Bulkhead of cabin,1,- .102,2,NaN
3,1,002,0018,0003,Panay,78,01,1,1,187,37,"S.P.Bray,Jr",01,3,23,1190,2,14,NaN,Bulkhead of cabin,1,- .102,2,NaN
4,1,002,0018,0003,Panay,78,01,1,1,187,37,"S.P.Bray,Jr",01,3,23,1190,2,14,NaN,Bulkhead of cabin,1,- .102,2,NaN


In [5]:
data_raw.data.c99_voyage.head()

,sentinal,reel_no,journal_no,frame_start,from_city,to_city
0,2,002,0018,0014,Boston,Rio de Janeiro
1,2,002,0018,0014,Boston,Rio de Janeiro
2,2,002,0018,0014,Boston,Rio de Janeiro
3,2,002,0018,0014,Boston,Rio de Janeiro
4,2,002,0018,0014,Boston,Rio de Janeiro


In [6]:
data_raw.data.c99_daily.head()

,sentinal,reel_no,journal_no,frame_start,frame,year,month,day,distance,lat_deg_an,lat_min_an,lat_hemis_an,lon_deg_an,lon_min_an,lon_hemis_an,lat_deg_on,lat_min_on,lat_hemis_on,lon_deg_of,lon_min_of,lon_hemis_of,current_speed,current_direction
0,3,002,0018,0014,0015,1878,10,20,NaN,<NA>,<NA>,NaN,<NA>,<NA>,NaN,42,20,N,66,30,W,0.1,E
1,3,002,0018,0014,0015,1878,10,20,NaN,<NA>,<NA>,NaN,<NA>,<NA>,NaN,42,20,N,66,30,W,0.1,E
2,3,002,0018,0014,0015,1878,10,20,NaN,<NA>,<NA>,NaN,<NA>,<NA>,NaN,42,20,N,66,30,W,0.1,E
3,3,002,0018,0014,0015,1878,10,20,NaN,<NA>,<NA>,NaN,<NA>,<NA>,NaN,42,20,N,66,30,W,0.1,E
4,3,002,0018,0014,0015,1878,10,20,NaN,<NA>,<NA>,NaN,<NA>,<NA>,NaN,42,20,N,66,30,W,0.1,E


In [7]:
data_raw.data.c99_data4.head()

,sentinal,reel_no,journal_no,frame_start,frame,year,month,day,time_ind,hour,ship_speed,compass_ind,ship_course_compass,compass_correction,ship_course_true,wind_dir_mag,wind_dir_true,wind_force,barometer,temp_ind,attached_thermometer,air_temperature,wet_bulb_temperature,sea_temperature,present_weather,clouds,sky_clear,sea_state
0,4,002,0018,0014,0015,1878,10,20,1,2,8.5,NaN,EXS,<NA>,NaN,WSW,NaN,06,2960,1,5.8,NaN,NaN,NaN,BOC,CU,5,R
1,4,002,0018,0014,0015,1878,10,20,1,4,8.5,NaN,EXS,<NA>,NaN,WSW,NaN,06,2960,1,5.6,NaN,NaN,NaN,BOC,SC,3,R
2,4,002,0018,0014,0015,1878,10,20,1,6,8.5,NaN,EXS,<NA>,NaN,W,NaN,06,2962,1,5.6,4.8,NaN,5.2,OCG,SC,0,R
3,4,002,0018,0014,0015,1878,10,20,1,8,8.0,NaN,EXS,<NA>,NaN,W,NaN,06,2964,1,5.6,4.8,NaN,5.2,CG,SC,0,R
4,4,002,0018,0014,0015,1878,10,20,1,10,8.5,NaN,EXS,<NA>,NaN,W,NaN,06,2969,1,5.7,4.8,NaN,5.0,BC,SC,2,L


In [8]:
data_raw.data.c99_data5.head()

,sentinal,reel_no,journal_no,frame_start,frame,year,month,day,time_ind,hour,ship_speed,compass_ind,ship_course_compass,blank,ship_course_true,wind_dir_mag,wind_dir_true,wind_force,barometer,temp_ind,attached_thermometer,air_temperature,wet_bulb_temperature,sea_temperature,present_weather,clouds,sky_clear,sea_state,compass_correction_ind,compass_correction,compass_correction_dir
0,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN


Now that we have separated the c99 data into the different sections, we see that this deck is composed of two types of data, which are the same:
    
    - c99_data4
    - c99_data5
    
Both sections have the same name in variables. To map the correct section into the CDM it is necessary to impose a filter on the sections composed only of NaN data. 
The problem is that we dont know which years in the time series will have a section c99_data4 and which will have a c99_data5

> Note that this solution of excluding one section, will only work for decks from which sections are exclusive: Among the sections listed in the block, only one of them appears in every report.


We can now use the `"icoads_r300_d704"` model to map the raw data to the Common Data Model [glamod/common_data_model](https://www.github.com/glamod/common_data_model). The function `map_model` from the `cdm_mapper` module contains all the functions for the model to convert variables to the correct units and/or specification following the [CDM Documentation](https://github.com/glamod/common_data_model/blob/master/cdm_latest.pdf).

To run the data model we need three things:

- raw data (the data we just read above)
- attributes of the raw data (sections and column names)
- the name of the model

In [9]:
name_of_model = "icoads_r300_d704"

cdm_dict = cdm.map_model(
    data_raw.data,
    name_of_model,
)

2024-10-09 07:50:37,325 - root - INFO - init basic configure of logging success
2024-10-09 07:50:37,373 - cdm_reader_mapper.cdm_mapper.mapper - WARNING - Could not convert    core   
     YR VS
0  1878  3
1  1878  3
2  1878  3
3  1878  3
4  1878  3 to frame.


Now, have we succeeded in writing some of the data to the CDM format?

We were looking to write the following data 

### Header section

 - Platform type and sub type
 - primary station id: original ship names
 - Longitude and Latitudes: converted from Degrees Minutes and Hemisphere to Decimal degrees
 - Location accuracy
 
 
### Observations tables

- `Observations-at`: latitude, longitude and location precision
- `Observations-dpt`: latitude, longitude and location precision
- `Observations-slp`: latitude, longitude and location precision
     - z_coordinate_type: Barometer height in feet converted to m.
     - original units: written in the CDM code format

- `Observations-sst`: latitude, longitude and location precision
- `Observations-wbt`: latitude, longitude and location precision
- `Observations-wd`: latitude, longitude and location precision
- `Observations-ws`: latitude, longitude and location precision


In [10]:
data = cdm_dict["header"]["data"]
data.head()

,source_record_id,platform_type,observing_programme,height_of_station_above_sea_level,report_type,primary_station_id,report_time_quality,primary_station_id_scheme,height_of_station_above_local_ground,history,report_meaning_of_timestamp,report_duration,source_id,station_speed,report_quality,duplicate_status,platform_sub_type,report_time_accuracy,record_timestamp,longitude,station_type,crs,latitude,location_quality,station_record_number,report_timestamp,application_area,location_accuracy,station_course,report_id,station_name
0,020N16,2,"[5, 7, 56]",0,0,Panay,2,8,0,2024-10-09 06:50:37. Initial conversion from I...,2,11,ICOADS-3-0-0T-125-704-1878-10,4.11552,0,4,26,3600,2024-10-09 06:50:37.381761+00:00,-68.41,2,0,42.28,0,1,1878-10-20 06:00:00,"[1, 7, 10, 11]",NaN,90,ICOADS-30-020N16,Panay
1,020N1P,2,"[5, 7, 56]",0,0,Panay,2,8,0,2024-10-09 06:50:37. Initial conversion from I...,2,11,ICOADS-3-0-0T-125-704-1878-10,4.11552,0,4,26,3600,2024-10-09 06:50:37.381761+00:00,-68.03,2,0,42.31,0,1,1878-10-20 08:00:00,"[1, 7, 10, 11]",NaN,90,ICOADS-30-020N1P,Panay
2,020N25,2,"[5, 7, 56]",0,0,Panay,2,8,0,2024-10-09 06:50:37. Initial conversion from I...,2,11,ICOADS-3-0-0T-125-704-1878-10,4.11552,0,4,26,3600,2024-10-09 06:50:37.381761+00:00,-67.64,2,0,42.33,0,1,1878-10-20 10:00:00,"[1, 7, 10, 11]",NaN,90,ICOADS-30-020N25,Panay
3,020N2Q,2,"[5, 7, 56]",0,0,Panay,2,8,0,2024-10-09 06:50:37. Initial conversion from I...,2,11,ICOADS-3-0-0T-125-704-1878-10,4.11552,0,4,26,3600,2024-10-09 06:50:37.381761+00:00,-67.29,2,0,42.35,0,1,1878-10-20 12:00:00,"[1, 7, 10, 11]",NaN,90,ICOADS-30-020N2Q,Panay
4,020N3A,2,"[5, 7, 56]",0,0,Panay,2,8,0,2024-10-09 06:50:37. Initial conversion from I...,2,11,ICOADS-3-0-0T-125-704-1878-10,4.11552,0,4,26,3600,2024-10-09 06:50:37.381761+00:00,-66.90,2,0,42.37,0,1,1878-10-20 14:00:00,"[1, 7, 10, 11]",NaN,90,ICOADS-30-020N3A,Panay


We now show an example of Lat and Lon

In [11]:
data.latitude.head(), data.longitude.head()

(0    42.28
 1    42.31
 2    42.33
 3    42.35
 4    42.37
 Name: latitude, dtype: float64,
 0   -68.41
 1   -68.03
 2   -67.64
 3   -67.29
 4   -66.90
 Name: longitude, dtype: float64)

In [12]:
data_raw.data.c99_daily[
    [
        "lat_deg_on",
        "lat_min_on",
        "lat_hemis_on",
        "lon_deg_of",
        "lon_min_of",
        "lon_hemis_of",
    ]
].head()

,lat_deg_on,lat_min_on,lat_hemis_on,lon_deg_of,lon_min_of,lon_hemis_of
0,42,20,N,66,30,W
1,42,20,N,66,30,W
2,42,20,N,66,30,W
3,42,20,N,66,30,W
4,42,20,N,66,30,W


This has been successfully converted to Decimal degrees with the right (-) for each hemisphere. 


Now for the SLP we have other information:

In [13]:
data_raw.data.c99_journal[["baro_type", "baro_height", "baro_units"]].head()

,baro_type,baro_height,baro_units
0,2,14,1
1,2,14,1
2,2,14,1
3,2,14,1
4,2,14,1


Baro type original code table

```
{
	"1":"aneroid",
	"2":"mercurial"
}
```
Baro units original code table. It has been left like this:

```
{
	"1":"inches",
	"2":"millimeters",
	"3":"millibars",
	"4":"unable to determine",
	"5":"Paris inches"
}
```

Our CDM table will be
```
{
  "1":1001,
  "2":1002,
  "3":1003,
  "4":9999,
  "5":1005
}
```

9999 will be the `"fill_value": 9999` that indicates to the CDM-mapper that these are NaN values.


In [14]:
data_obs = cdm_dict["observations-slp"]["data"]
data_obs.head()

,advanced_homogenisation,original_precision,exposure_of_sensor,date_time,quality_flag,traceability,numerical_precision,date_time_meaning,advanced_uncertainty,location_precision,data_policy_licence,source_id,value_significance,z_coordinate_type,longitude,crs,latitude,conversion_flag,advanced_qc,spatial_representativeness,sensor_automation_status,report_id,observation_id,observation_duration,processing_level,observation_value,units,original_units,observed_variable,original_value,conversion_method,z_coordinate,observation_height_above_station_surface
0,0,NaN,3,1878-10-20 06:00:00,2,2,NaN,2,0,NaN,0,ICOADS-3-0-0T-125-704-1878-10,2,0,-68.41,0,42.28,0,0,3,5,ICOADS-30-020N16,ICOADS-30-020N16-SLP,8,3,99610.0,32,1001,58,996.1,7,4.27,4.27
1,0,NaN,3,1878-10-20 08:00:00,2,2,NaN,2,0,NaN,0,ICOADS-3-0-0T-125-704-1878-10,2,0,-68.03,0,42.31,0,0,3,5,ICOADS-30-020N1P,ICOADS-30-020N1P-SLP,8,3,99630.0,32,1001,58,996.3,7,4.27,4.27
2,0,NaN,3,1878-10-20 10:00:00,2,2,NaN,2,0,NaN,0,ICOADS-3-0-0T-125-704-1878-10,2,0,-67.64,0,42.33,0,0,3,5,ICOADS-30-020N25,ICOADS-30-020N25-SLP,8,3,99690.0,32,1001,58,996.9,7,4.27,4.27
3,0,NaN,3,1878-10-20 12:00:00,2,2,NaN,2,0,NaN,0,ICOADS-3-0-0T-125-704-1878-10,2,0,-67.29,0,42.35,0,0,3,5,ICOADS-30-020N2Q,ICOADS-30-020N2Q-SLP,8,3,99760.0,32,1001,58,997.6,7,4.27,4.27
4,0,NaN,3,1878-10-20 14:00:00,2,2,NaN,2,0,NaN,0,ICOADS-3-0-0T-125-704-1878-10,2,0,-66.90,0,42.37,0,0,3,5,ICOADS-30-020N3A,ICOADS-30-020N3A-SLP,8,3,99920.0,32,1001,58,999.2,7,4.27,4.27
